In [2]:
import plyvel
import json


db = plyvel.DB('../hydra-data/ipns-export-M01')
num = 0
for k, v in db:
    num += 1
print(num)

Error: b'NotFound: ../hydra-data/ipns-export-M01/LOCK: No such file or directory'

In [3]:

    obj = json.loads(v)
    print(v)
    if obj['NameResolveTime']:
        print(obj['NameResolveTime'])
    break

NameError: name 'v' is not defined